# İmport Modules

In [ ]:
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
import seaborn as sns



from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

# Train & Test Read

In [ ]:
train = pd.read_csv("../input/digit-recognizer/train.csv")
print(train.shape)
train.head()

In [ ]:
test= pd.read_csv("../input/digit-recognizer/test.csv")
print(test.shape)
test.head()

# X,Y Split

In [ ]:
Y_train = train["label"]
# Drop 'label' column
X_train = train.drop("label",axis = 1) 

# Number is Value Counts

In [ ]:
plt.figure(figsize=(15,7))
g = sns.countplot(Y_train, palette="icefire")
plt.title("Number of digit classes")
Y_train.value_counts();

In [ ]:
fig,ax=plt.subplots(2,3,figsize=(15,10))

for i in range(2):
    for j in range(3):
        select=np.random.randint(0,len(train))
        img = train.iloc[select,1:].values
        img = img.reshape((28,28))
        ax[i][j].imshow(img)
        ax[i][j].set_title(train.iloc[select,0],fontsize=20)
        #ax[i][j].axis("off")
        #plt.show()

In [ ]:
fig,ax=plt.subplots(2,3,figsize=(15,10))

for i in range(2):
    for j in range(3):
        select=np.random.randint(0,len(train))
        img = train.iloc[select,1:].values
        img = img.reshape((28,28))
        ax[i][j].imshow(img,cmap="gray")
        ax[i][j].set_title(train.iloc[select,0],fontsize=20)
        ax[i][j].axis("off")

# Normalize Data

In [ ]:
X_train = X_train / 255.0
test = test / 255.0
print("x_train shape: ",X_train.shape)
print("test shape: ",test.shape)

In [ ]:
X_train.shape

# Reshape Data

In [ ]:
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)
print("x_train shape: ",X_train.shape)
print("test shape: ",test.shape)

In [ ]:
Y_train[:5]

# Label Encoding

In [ ]:
Y_train = to_categorical(Y_train, num_classes = 10)
Y_train[:5]

# Train Test Split

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=2)
print("x_train shape",X_train.shape)
print("x_test shape",X_val.shape)
print("y_train shape",Y_train.shape)
print("y_test shape",Y_val.shape)

In [ ]:
plt.imshow(X_train[10][:,:,0],cmap='gray')
plt.show()

# Create Model

In [ ]:
model = Sequential()
#
model.add(Conv2D(filters = 8, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
#
model.add(Conv2D(filters = 16, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [ ]:
model.summary()

In [ ]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
epochs = 10
batch_size = 250

# Data Replication

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=5,
        zoom_range = 0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=False,
        vertical_flip=False)

datagen.fit(X_train)

In [ ]:
history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val), steps_per_epoch=X_train.shape[0] // batch_size)

In [ ]:
Y_train[:5]

# Evaluate The Model

In [ ]:
plt.plot(history.history['val_loss'], color='b', label="validation loss")
plt.title("Test Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
Y_pred = model.predict(X_val)
Y_pred_classes = np.argmax(Y_pred,axis = 1)

In [ ]:
fig,ax=plt.subplots(2,3,figsize=(15,10))

for i in range(2):
    for j in range(3):
        select_number=np.random.randint(0,len(X_val.shape))
        img = X_val[select_number]
        img = img.reshape((28,28))
        ax[i][j].imshow(img)
        ax[i][j].set_title(Y_pred_classes[select_number],fontsize=20)
        #ax[i][j].axis("off")
        #plt.show()

# Confusion Matrix

In [ ]:
Y_pred = model.predict(X_val)
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
Y_true = np.argmax(Y_val,axis = 1) 
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()